# 爬PTT地震文

* 需要的資訊：
    
    1. 發文時間
    2. 文章標題
    3. 文章內容
    4. 前100推推文內容
    5. 發文者ID
    6. 發文者IP位址
    7. 該文是否屬於地震文（dummy）
    8. 該文是否搶到地震爆文（dummy）
   
* 另外要寫的功能：

    1. 用IP位址反查地址資訊，回傳「縣市」（str）及經緯度（某種可以被餵進google map API的格式）
    2. 一天／一週當中PTT上站人數的pattern，輔以「當時發文者發文所處的時間」來控制error term

In [1]:
# -*- coding: utf-8 -*-

# 導入 模組(module) 
import requests 
# 導入 BeautifulSoup 模組(module)：解析HTML 語法工具
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

from multiprocessing import Pool #https://github.com/leVirve/CrawlerTutorial

/Applications/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#八卦版文章連結
url = "https://www.ptt.cc/bbs/Gossiping/index.html"
header_over18 = {"cookie":"over18=1"}
res = requests.get(url, headers = header_over18)
res

<Response [200]>

In [3]:
soup = BeautifulSoup(res.text, "html.parser")

# try to search keyword "地震"

In [4]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
resp = requests.get(search_endpoint_url, params={'q': '震'}, headers = header_over18)

In [5]:
soup = BeautifulSoup(resp.text, "html.parser")

In [6]:
# 找到全部地震文章的連結
links = soup.find_all("a href")

In [7]:
titles = soup.find_all("div", "title")

In [8]:
for title in titles:
    print(title.text, title.a["href"])
    link = "https://www.ptt.cc/" + title.a["href"]


[爆卦] 地震啦
 /bbs/Gossiping/M.1591921411.A.387.html

[爆卦] 南投地震
 /bbs/Gossiping/M.1591921396.A.FA1.html

[爆卦] 地震
 /bbs/Gossiping/M.1591921322.A.F87.html

[新聞] 幼獅被打斷腿供遊客合影 普丁震怒令調查
 /bbs/Gossiping/M.1591915588.A.72B.html

[問卦] 震撼彈 ! 道頓堀　不唸做  道頓ㄐㄩㄝˊ ?
 /bbs/Gossiping/M.1591871567.A.2A7.html

[問卦] 英國噴射推進飛機試飛現場有多震撼？
 /bbs/Gossiping/M.1591853488.A.476.html

[問卦] 震怒！媽媽在我的水餃裡加料！？
 /bbs/Gossiping/M.1591851760.A.894.html

Re: [問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591840928.A.FDC.html

Re: [問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591840584.A.E2D.html

[問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591839873.A.311.html

[問卦] 地震第一時間要做什麼才是正常人？
 /bbs/Gossiping/M.1591839848.A.0A8.html

[問卦] 地震時沒穿衣服怎麼辦？
 /bbs/Gossiping/M.1591839805.A.DF0.html

[爆卦] 地震！
 /bbs/Gossiping/M.1591839708.A.74D.html

[爆卦] 我愛你地震
 /bbs/Gossiping/M.1591839684.A.18F.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839641.A.DF7.html

[爆卦] 地震！
 /bbs/Gossiping/M.1591839618.A.C92.html

[問卦] 好震歐
 /bbs/Gossiping/M.1591839616.A.A62.html

地震
 /bbs/Gossiping/M.1591839606.A

In [9]:
article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
article_id

'1591839603'

In [10]:
# 進入搜尋頁面的第一篇文章
link = "https://www.ptt.cc/" + title.a["href"]

# 18禁
payload = {
    "from" : "/bbs/Gossiping/index.html",
    "yes"  : "yes"
}
rs = requests.session()
res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)

res_subpage = rs.get(link, verify = False)
soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda

In [11]:
print(soup_subpage.text)






[爆卦] 地震 - 看板 Gossiping - 批踢踢實業坊
















批踢踢實業坊
›
看板 Gossiping
關於我們
聯絡資訊




返回看板




作者fixjo (fixjo)看板Gossiping標題[爆卦] 地震時間Thu Jun 11 09:40:01 2020


停了

※註:有電視或媒體有報導者，請勿使用爆卦!
無重大八卦請勿使用此分類，否則視同濫用爆卦鬧板(文章退回、水桶6個月)
※政治類爆卦改為政治分類標題

未滿30繁體中文字 水桶3個月

-----
Sent from JPTT on my Samsung SM-G955F.

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 111.71.16.148 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1591839603.A.59B.html


本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。


  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');

  ga('create', 'UA-32365737-1', {
    cookieDomain: 'ptt.cc',
    legacyCookieDomain: 'ptt.cc'
  });
  ga('send', 'pageview');



(function(){window['__CF$cv$params']={r:'5a205036aa7ef09d',m:'bd8e469e98b7a4e53ea5a

In [12]:
#一篇文章的 作者、標題、時間、看板
header = soup_subpage.find_all("span", "article-meta-value")
print(header)

[<span class="article-meta-value">fixjo (fixjo)</span>, <span class="article-meta-value">Gossiping</span>, <span class="article-meta-value">[爆卦] 地震</span>, <span class="article-meta-value">Thu Jun 11 09:40:01 2020</span>]


In [13]:
print(header[0].text) #author
print(header[1].text) #board
print(header[2].text) #title
print(header[3].text) #date

fixjo (fixjo)
Gossiping
[爆卦] 地震
Thu Jun 11 09:40:01 2020


In [14]:
main_text = soup_subpage.find(id = "main-container").text
print(main_text)


作者fixjo (fixjo)看板Gossiping標題[爆卦] 地震時間Thu Jun 11 09:40:01 2020


停了

※註:有電視或媒體有報導者，請勿使用爆卦!
無重大八卦請勿使用此分類，否則視同濫用爆卦鬧板(文章退回、水桶6個月)
※政治類爆卦改為政治分類標題

未滿30繁體中文字 水桶3個月

-----
Sent from JPTT on my Samsung SM-G955F.

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 111.71.16.148 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1591839603.A.59B.html


本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。



In [15]:
#推文
main_text.split(sep = "html")[1].split(sep = "\n")[1:]

['', '', '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。', '']

In [16]:
# 把整個內容切割透過 "-- " 切割成2個陣列
pre_text = main_text.split('--')[0]
    
# 把每段文字 根據 '\n' 切開
texts = pre_text.split('\n')
# 如果你爬多篇你會發現 
contents = texts[2:]
# 內容
content = '\n'.join(contents)

print('內容：'+content)

內容：

停了

※註:有電視或媒體有報導者，請勿使用爆卦!
無重大八卦請勿使用此分類，否則視同濫用爆卦鬧板(文章退回、水桶6個月)
※政治類爆卦改為政治分類標題

未滿30繁體中文字 水桶3個月




In [17]:
IP_address = main_text.split('來自:')[1].split("\n")[0]
IP_address

' 111.71.16.148 (臺灣)'

```
f"{article_id}" = {
    "Date" : ""
    "Title" : ""
    "Author" : ""
    "IP" : ""
    "Content" : 
}
```

In [18]:
f"{article_id}"

'1591839603'

In [19]:
exec_str = "M" + f"{article_id}"
exec_str

'M1591839603'

In [20]:
exec(exec_str+" = {}", globals())

In [21]:
type(M1590366890)

NameError: name 'M1590366890' is not defined

In [22]:
M1590366890[1] = "01"
M1590366890[2] = "02"

NameError: name 'M1590366890' is not defined

In [23]:
exec(f"{exec_str}", globals())

## 實作：把搜尋頁第一頁的文章都爬下來，存在一個dict內

In [24]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(1, 52+1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        link = "https://www.ptt.cc/" + title.a["href"]
        article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

# 八卦版前一百頁

In [175]:
# 八卦版前一百頁
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(39284, 39184, -1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/index" + str(page_count) + ".html"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        try:
            link = "https://www.ptt.cc/" + title.a["href"]
        except:
            pass
        #article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

In [25]:
title

<div class="title">
<a href="/bbs/Gossiping/M.1579190556.A.819.html">[問卦] 震撼彈！！！！今天學測！！！</a>
</div>

In [26]:
header[0]

<span class="article-meta-value">paulabxz123 (優文大師)</span>

In [27]:
author = header[0].text
authorID = author.split(sep = " (")[0]
#print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])

In [28]:
authorID

'paulabxz123'

In [29]:
page_count

52

In [31]:
len(earthquake_dict)

1040

In [32]:
earthquake_dict.keys()

dict_keys(['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25', 'M26', 'M27', 'M28', 'M29', 'M30', 'M31', 'M32', 'M33', 'M34', 'M35', 'M36', 'M37', 'M38', 'M39', 'M40', 'M41', 'M42', 'M43', 'M44', 'M45', 'M46', 'M47', 'M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62', 'M63', 'M64', 'M65', 'M66', 'M67', 'M68', 'M69', 'M70', 'M71', 'M72', 'M73', 'M74', 'M75', 'M76', 'M77', 'M78', 'M79', 'M80', 'M81', 'M82', 'M83', 'M84', 'M85', 'M86', 'M87', 'M88', 'M89', 'M90', 'M91', 'M92', 'M93', 'M94', 'M95', 'M96', 'M97', 'M98', 'M99', 'M100', 'M101', 'M102', 'M103', 'M104', 'M105', 'M106', 'M107', 'M108', 'M109', 'M110', 'M111', 'M112', 'M113', 'M114', 'M115', 'M116', 'M117', 'M118', 'M119', 'M120', 'M121', 'M122', 'M123', 'M124', 'M125', 'M126', 'M127', 'M128', 'M129', 'M130', 'M131', 'M132', 'M133', 'M134', 'M135', 'M136', 'M137', '

In [36]:
earthquake_dict["M3"]

{'Author': 'AntitheApple (蘋果? 那是什麼好吃嗎)',
 'Board': 'Gossiping',
 'Title': '[爆卦] 地震',
 'Date': 'Fri Jun 12 08:21:58 2020',
 'Content': '居然有地震\n\n\n\n發震時間：2020/06/12 08:21:35\n\n位置：北緯 24.01度，東經 121.69度即在花蓮縣政府東偏北方 7.8 公里 ，位於花蓮縣近海\n\n地震深度：30.2公里\n\n芮氏規模：4.7\n\n\nhttp://i.imgur.com/eIpqMFc.jpg\n',
 'Full IP': ' 114.44.214.137 (臺灣)',
 'IP': '114.44.214.137',
 'Push': ['※ 編輯: AntitheApple (114.44.214.137 臺灣), 06/12/2020 08:22:21',
  '噓 KLGlikeshit: ？06/12 08:22',
  '推 starjack2001: 東華小搖06/12 08:22',
  '噓 ivorysoap: 美國無感06/12 08:22',
  '推 momoni2012: 台中有感06/12 08:22',
  '推 night30231: 搖06/12 08:22',
  '推 yukime46: 東華超抖06/12 08:22',
  '噓 zxc2331189: 掰06/12 08:22',
  '推 legendyelon: 還真的有地震06/12 08:22',
  '推 B08092005: 台中有感06/12 08:22',
  '推 vs3: 幹你贏了06/12 08:22',
  '推 dododododora: 桃園無感 06/12 08:22',
  '推 Walter0710: 強06/12 08:22',
  '推 lulu613: 好大！06/12 08:22',
  '推 naiyomin: 台中有感 06/12 08:22',
  '推 mina2ii: 台中有感 06/12 08:22',
  '→ paul2chiu: 無感 06/12 08:22',
  '推 hsin0918: 中部有感 06/12 08:22',
 

In [37]:
IP_shorten = IP_address.split(sep = " ")[1]
IP_shorten

'114.137.59.180'

# Save Dictionary

In [38]:
# Save
np.save("earthquake_dict_0612", earthquake_dict)

# Load
#read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

In [39]:
earthquake_dict["M20"]

{'Author': 'fixjo (fixjo)',
 'Board': 'Gossiping',
 'Title': '[爆卦] 地震',
 'Date': 'Thu Jun 11 09:40:01 2020',
 'Content': '\n\n停了\n\n※註:有電視或媒體有報導者，請勿使用爆卦!\n無重大八卦請勿使用此分類，否則視同濫用爆卦鬧板(文章退回、水桶6個月)\n※政治類爆卦改為政治分類標題\n\n未滿30繁體中文字 水桶3個月\n\n',
 'Full IP': ' 111.71.16.148 (臺灣)',
 'IP': '111.71.16.148',
 'Push': ['', '', '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。', '']}